## 4.3 Householder transformations

In [ ]:
import numpy as np
from scripts.lu import backward

We implement the QR factorization using Householder transformations and store the Householder vectors in another matrix.

In [ ]:
def qr_householder(A):
    n, m = A.shape
    V = np.zeros_like(A)
    
    for i in range(m):
        V[i:, i] = A[i:, i]
        ei = np.zeros(n - i, dtype=A.dtype)
        ei[0] = 1.0
        V[i:, i] += np.sign(A[i, i]) * np.linalg.norm(V[i:, i]) * ei
        V[i:, i] /= np.linalg.norm(V[i:, i])
        for k in range(i, m):
            A[i:, k] -= 2 * np.inner(V[i:, i], A[i:, k]) * V[i:, i]
    return V

*Note: We already distinguish here between the number of rows and columns in order to cover the case $n>m$ later on.

In order to apply the QR decomposition with Householder transformations efficiently, we still have to implement the application of $Q^T$ without setting up the matrix.

In [ ]:
def QT_apply(V, b):
    for i in range(b.shape[0]):
        b[i:] -= 2 * np.inner(V[i:, i], b[i:]) * V[i:, i]
    return None

#### Example 4.17 (QR factorisation with Householder transformations)

We apply the QR factorization with Householder transformations to the same linear system with which we tested the QR factorization using Gram-Schmidt orthogonalization.

In [ ]:
A = np.array([[1,    1,    1   ],
              [0.01, 0,    0.01],
              [0,    0.01, 0.01]], dtype=np.half)
b = np.array([1, 0, 0.02], dtype=np.half)
x_ex = np.array([-1, 1, 1])

In [ ]:
V = qr_householder(A)
print(V)

In [ ]:
QT_apply(V, b)
x = backward(A, b)
print(x)

We see that this is almost the exact solution. In fact, the relative error is only

In [ ]:
rel_err = np.linalg.norm(x - x_ex) / np.linalg.norm(x_ex)
print(f'||x - x_ex|| / ||x_ex|| = {rel_err:.3e}')

We can also calculate the matrix $Q^T$ from the Householder vectors to check the orthogonality of $Q$:

In [ ]:
n, m = V.shape
QT = np.eye(m, dtype=V.dtype)
for i in range(m):
    S = np.eye(m, dtype=V.dtype)
    S[i:, i:] = np.eye(m - i, dtype=V.dtype) - 2 * np.outer(V[i:, i], V[i:, i])
    QT = S @ QT

print(QT)

In [ ]:
print(np.linalg.norm(QT @ QT.T - np.eye(3, dtype=np.single), 2))

In [ ]:
print(QT @ QT.T)